# PySparkでデータ前処理100本ノックをやってみる

* 参考
    * [PySparkでCSVファイルを読み込む方法](https://docs.kanaries.net/ja/articles/read-csv-dataframe)
    * [PySparkでparquetファイルを読み込む方法](https://data-analysis-stats.jp/spark/pyspark%E3%81%A7%E3%83%87%E3%83%BC%E3%82%BF%E8%AA%AD%E3%81%BF%E8%BE%BC%E3%81%BF/)
    * [PolarsでCSVファイルをParquetに変換する方法](https://medium.com/@umesh.nagar92x/efficient-conversion-of-massive-csv-files-to-parquet-format-using-pandas-dask-duck-db-and-polars-5c998c47b43d)

In [10]:
from glob import glob

import polars as pl
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql import functions as F

# Create a SparkSession。pythonからsparkを使う場合、セッションの作成が必要。
spark = SparkSession.builder.appName("Testing PySpark Example").getOrCreate()

# デフォルトのログレベルだと大量にログが出力されるので限定する。
spark.sparkContext.setLogLevel("ERROR")

## Polarsを用いて入力ファイル(csv)をParquetに変換する
PySparkはcsvを読むのが苦手なので、Parquetファイルに変換する

In [28]:
csv_files = glob('../../../100knocks-preprocess/docker/work/data/*.csv')
csv_files

['../../../100knocks-preprocess/docker/work/data/store.csv',
 '../../../100knocks-preprocess/docker/work/data/receipt.csv',
 '../../../100knocks-preprocess/docker/work/data/category.csv',
 '../../../100knocks-preprocess/docker/work/data/product.csv',
 '../../../100knocks-preprocess/docker/work/data/customer.csv',
 '../../../100knocks-preprocess/docker/work/data/geocode.csv']

In [29]:
dtypes = {
    'customer_id': str,
    'gender_cd': str,
    'postal_cd': str,
    'application_store_cd': str,
    'status_cd': str,
    'category_major_cd': str,
    'category_medium_cd': str,
    'category_small_cd': str,
    'product_cd': str,
    'store_cd': str,
    'prefecture_cd': str,
    'tel_no': str,
    'postal_cd': str,
    'street': str,
    'application_date': str,
    'birth_day': pl.Date
}

# それぞれcsvを読み込んでからparquetとして出力
for file in csv_files:
    df = pl.read_csv(file, dtypes=dtypes)

    # fileの絶対パスの拡張子をparquetにして出力
    output_path = file.split(".csv")[0] + ".parquet"
    df.write_parquet(output_path)


## PySparkでParquetファイルを読み込み

In [38]:
df_receipt = (
    spark.read
    .parquet("../../../100knocks-preprocess/docker/work/data/receipt.parquet")
)

In [42]:
df_receipt.dtypes

[('sales_ymd', 'bigint'),
 ('sales_epoch', 'bigint'),
 ('store_cd', 'string'),
 ('receipt_no', 'bigint'),
 ('receipt_sub_no', 'bigint'),
 ('customer_id', 'string'),
 ('product_cd', 'string'),
 ('quantity', 'bigint'),
 ('amount', 'bigint')]

# 100本ノック

---
> P-001: レシート明細データ（df_receipt）から全項目の先頭10件を表示し、どのようなデータを保有しているか目視で確認せよ。

In [44]:
df_receipt.show(10)

+---------+-----------+--------+----------+--------------+--------------+----------+--------+------+
|sales_ymd|sales_epoch|store_cd|receipt_no|receipt_sub_no|   customer_id|product_cd|quantity|amount|
+---------+-----------+--------+----------+--------------+--------------+----------+--------+------+
| 20181103| 1541203200|  S14006|       112|             1|CS006214000001|P070305012|       1|   158|
| 20181118| 1542499200|  S13008|      1132|             2|CS008415000097|P070701017|       1|    81|
| 20170712| 1499817600|  S14028|      1102|             1|CS028414000014|P060101005|       1|   170|
| 20190205| 1549324800|  S14042|      1132|             1|ZZ000000000000|P050301001|       1|    25|
| 20180821| 1534809600|  S14025|      1102|             2|CS025415000050|P060102007|       1|    90|
| 20190605| 1559692800|  S13003|      1112|             1|CS003515000195|P050102002|       1|   138|
| 20181205| 1543968000|  S14024|      1102|             2|CS024514000042|P080101005|       

---
> P-002: レシート明細データ（df_receipt）から売上年月日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示せよ。

In [45]:
df_receipt.select(
    "sales_ymd",
    "customer_id",
    "product_cd",
    "amount"
).show(10)

+---------+--------------+----------+------+
|sales_ymd|   customer_id|product_cd|amount|
+---------+--------------+----------+------+
| 20181103|CS006214000001|P070305012|   158|
| 20181118|CS008415000097|P070701017|    81|
| 20170712|CS028414000014|P060101005|   170|
| 20190205|ZZ000000000000|P050301001|    25|
| 20180821|CS025415000050|P060102007|    90|
| 20190605|CS003515000195|P050102002|   138|
| 20181205|CS024514000042|P080101005|    30|
| 20190922|CS040415000178|P070501004|   128|
| 20170504|ZZ000000000000|P071302010|   770|
| 20191010|CS027514000015|P071101003|   680|
+---------+--------------+----------+------+
only showing top 10 rows



---
> P-003: レシート明細データ（df_receipt）から売上年月日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、  
> 10件表示せよ。ただし、sales_ymdをsales_dateに項目名を変更しながら抽出すること。

In [48]:
df_receipt.select(
    F.col("sales_ymd").alias("sales_date"),
    "customer_id",
    "product_cd",
    "amount"
).show(10)

+----------+--------------+----------+------+
|sales_date|   customer_id|product_cd|amount|
+----------+--------------+----------+------+
|  20181103|CS006214000001|P070305012|   158|
|  20181118|CS008415000097|P070701017|    81|
|  20170712|CS028414000014|P060101005|   170|
|  20190205|ZZ000000000000|P050301001|    25|
|  20180821|CS025415000050|P060102007|    90|
|  20190605|CS003515000195|P050102002|   138|
|  20181205|CS024514000042|P080101005|    30|
|  20190922|CS040415000178|P070501004|   128|
|  20170504|ZZ000000000000|P071302010|   770|
|  20191010|CS027514000015|P071101003|   680|
+----------+--------------+----------+------+
only showing top 10 rows



---
> P-004: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"

In [51]:
df_receipt.select(
    "sales_ymd",
    "customer_id",
    "product_cd",
    "amount"
).filter(
    F.col("customer_id") == "CS018205000001"
).show()

+---------+--------------+----------+------+
|sales_ymd|   customer_id|product_cd|amount|
+---------+--------------+----------+------+
| 20180911|CS018205000001|P071401012|  2200|
| 20180414|CS018205000001|P060104007|   600|
| 20170614|CS018205000001|P050206001|   990|
| 20170614|CS018205000001|P060702015|   108|
| 20190216|CS018205000001|P071005024|   102|
| 20180414|CS018205000001|P071101002|   278|
| 20190226|CS018205000001|P070902035|   168|
| 20190924|CS018205000001|P060805001|   495|
| 20190226|CS018205000001|P071401020|  2200|
| 20180911|CS018205000001|P071401005|  1100|
| 20190216|CS018205000001|P040101002|   218|
| 20190924|CS018205000001|P091503001|   280|
+---------+--------------+----------+------+



---
> P-005: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、  
> 売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上

In [53]:
df_receipt.select(
    "sales_ymd",
    "customer_id",
    "product_cd",
    "amount"
).filter(
    (F.col("customer_id") == "CS018205000001") & (F.col("amount") >= 1000 )
).show()

+---------+--------------+----------+------+
|sales_ymd|   customer_id|product_cd|amount|
+---------+--------------+----------+------+
| 20180911|CS018205000001|P071401012|  2200|
| 20190226|CS018205000001|P071401020|  2200|
| 20180911|CS018205000001|P071401005|  1100|
+---------+--------------+----------+------+



---
> P-006: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上数量（quantity）、売上金額（amount）の順に列を指定し、  
> 以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上または売上数量（quantity）が5以上

In [59]:
cols = ["sales_ymd","customer_id","product_cd","quantity","amount"]
df_receipt.select(
    cols
).filter(
    (F.col("customer_id") == "CS018205000001") &
    ((F.col("amount") >= 1000) | (F.col("quantity") >= 5))
).show()

+---------+--------------+----------+--------+------+
|sales_ymd|   customer_id|product_cd|quantity|amount|
+---------+--------------+----------+--------+------+
| 20180911|CS018205000001|P071401012|       1|  2200|
| 20180414|CS018205000001|P060104007|       6|   600|
| 20170614|CS018205000001|P050206001|       5|   990|
| 20190226|CS018205000001|P071401020|       1|  2200|
| 20180911|CS018205000001|P071401005|       1|  1100|
+---------+--------------+----------+--------+------+



---
> P-007: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、  
> 以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上2,000以下

In [60]:
cols = ["sales_ymd","customer_id","product_cd","amount"]
df_receipt.select(
    cols
).filter(
    (F.col("customer_id") == "CS018205000001") &
    (F.col("amount").between(1000, 2000))
).show()

+---------+--------------+----------+------+
|sales_ymd|   customer_id|product_cd|amount|
+---------+--------------+----------+------+
| 20180911|CS018205000001|P071401005|  1100|
+---------+--------------+----------+------+

